# 📚 Método de Wang-Mendel para Geração Automática de Regras Fuzzy

**Aula 3 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 🎯 Objetivo

Neste notebook, vamos aplicar o **Método de Wang-Mendel (1992)** para **gerar automaticamente regras fuzzy** a partir de dados.

### Problema: Aproximar f(x) = sin(x) + 0.1x

- **Domínio**: x ∈ [0, 2π]
- **Função não-linear** (mais complexa que a reta!)
- **Objetivo**: Gerar regras fuzzy que aproximem a função

---

## 📖 Referência Original

**Wang, L. X., & Mendel, J. M. (1992)**. "Generating fuzzy rules by learning from examples."  
*IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

- 🏆 **Mais de 5000 citações**
- 📊 Aplicações: Controle fuzzy, séries temporais, aproximação de funções
- 🧠 Algoritmo simples mas poderoso: **um único passo pelos dados**

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] -q

print('✅ pyfuzzy-toolbox instalado com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendelLearning
from fuzzy_systems.inference import MamdaniSystem

%matplotlib inline

# Configuração
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 🔍 Passo 1: O Problema - Aproximar f(x) = sin(x) + 0.1x

Vamos gerar dados de uma função **não-linear** e aplicar Wang-Mendel!

In [ ]:
# ============================================================================
# 1. GERAR DADOS DO PROBLEMA - CORRIGIDO
# ============================================================================

# Função verdadeira
def funcao_real(x):
    """f(x) = sin(x) + 0.1x"""
    return np.sin(x) + 0.1 * x

# Gerar dados de treinamento - ATENÇÃO AO .reshape(-1, 1)
n_samples = 50
X_train = np.linspace(0, 2*np.pi, n_samples).reshape(-1, 1)  # Shape (50, 1) ✓
y_train = funcao_real(X_train).reshape(-1, 1)  # Shape (50, 1) ✓
y_train += np.random.normal(0, 0.05, y_train.shape)

# Dados de teste
X_test = np.linspace(0, 2*np.pi, 200).reshape(-1, 1)  # Shape (200, 1) ✓
y_true = funcao_real(X_test).flatten()  # Para plotting

# Verificar shapes
print('🔍 Verificando Shapes:')
print(f'   X_train.shape = {X_train.shape}  # Deve ser (50, 1)')
print(f'   y_train.shape = {y_train.shape}  # Deve ser (50, 1)')
print(f'   X_test.shape = {X_test.shape}   # Deve ser (200, 1)')

print('\n📊 Configuração do Problema:')
print(f'   • Função: f(x) = sin(x) + 0.1x')
print(f'   • Domínio: x ∈ [0, 2π] = [0, {2*np.pi:.3f}]')
print(f'   • Imagem: y ∈ [{y_train.min():.3f}, {y_train.max():.3f}]')
print(f'   • Amostras de treino: {n_samples}')
print(f'   • Amostras de teste: {len(X_test)}')

In [ ]:
# ============================================================================
# 2. CRIAR SISTEMA FUZZY MAMDANI COM 11 PARTIÇÕES
# ============================================================================

print('\n🔧 Criando Sistema Fuzzy Mamdani com 11 Partições...')

# Criar sistema
sistema = MamdaniSystem(name='WangMendelSinApprox')

# Domínios
x_min, x_max = 0, 2*np.pi
y_min, y_max = y_train.min() - 0.1, y_train.max() + 0.1

# Adicionar variável de entrada com 11 partições
sistema.add_input('x', (x_min, x_max))

# Criar 11 termos linguísticos triangulares sobrepostos
n_partitions = 11
partition_names = [
    'very_low', 'low_1', 'low_2', 'medium_low', 'medium_1', 'medium_2',
    'medium_high', 'high_1', 'high_2', 'very_high_1', 'very_high_2'
]

# Gerar partições uniformemente distribuídas
x_range = x_max - x_min
step = x_range / (n_partitions - 1)

for i, name in enumerate(partition_names):
    center = x_min + i * step
    left = max(x_min, center - step)
    right = min(x_max, center + step)
    
    # Primeiros e últimos termos são trapezoidais (fecham as pontas)
    if i == 0:
        params = [x_min, x_min, center, center + step]
        sistema.add_term('x', name, 'trapezoidal', params)
    elif i == n_partitions - 1:
        params = [center - step, center, x_max, x_max]
        sistema.add_term('x', name, 'trapezoidal', params)
    else:
        params = [left, center, right]
        sistema.add_term('x', name, 'triangular', params)

print(f'   ✓ Entrada "x" configurada com {n_partitions} termos linguísticos')

# Adicionar variável de saída com 11 partições
sistema.add_output('y', (y_min, y_max))

y_range = y_max - y_min
y_step = y_range / (n_partitions - 1)

output_names = [
    'very_low', 'low_1', 'low_2', 'medium_low', 'medium_1', 'medium_2',
    'medium_high', 'high_1', 'high_2', 'very_high_1', 'very_high_2'
]

for i, name in enumerate(output_names):
    center = y_min + i * y_step
    left = max(y_min, center - y_step)
    right = min(y_max, center + y_step)
    
    if i == 0:
        params = [y_min, y_min, center, center + y_step]
        sistema.add_term('y', name, 'trapezoidal', params)
    elif i == n_partitions - 1:
        params = [center - y_step, center, y_max, y_max]
        sistema.add_term('y', name, 'trapezoidal', params)
    else:
        params = [left, center, right]
        sistema.add_term('y', name, 'triangular', params)

print(f'   ✓ Saída "y" configurada com {n_partitions} termos linguísticos')

In [ ]:
# ============================================================================
# 3. TREINAR COM WANG-MENDEL
# ============================================================================

print('\n🤖 Treinando com Algoritmo Wang-Mendel...')

# Instanciar e treinar
wm = WangMendelLearning(sistema, X_train, y_train)
sistema_treinado = wm.fit(verbose=True)

# Estatísticas
stats = wm.get_training_stats()
print(f'\n📈 Estatísticas de Treinamento:')
print(f'   • Regras candidatas: {stats["candidate_rules"]}')
print(f'   • Regras finais: {stats["final_rules"]}')
print(f'   • Conflitos resolvidos: {stats["conflicts_resolved"]}')

In [ ]:
# ============================================================================
# 4. FAZER PREDIÇÕES E AVALIAR
# ============================================================================

print('\n🎯 Fazendo Predições...')

# Predição
y_pred = wm.predict(X_test).flatten()

# Métricas
mse = np.mean((y_pred - y_true)**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_pred - y_true))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2))

print(f'\n📊 Métricas de Performance:')
print(f'   • MSE (Mean Squared Error): {mse:.6f}')
print(f'   • RMSE (Root MSE): {rmse:.6f}')
print(f'   • MAE (Mean Absolute Error): {mae:.6f}')
print(f'   • R² Score: {r2:.6f}')


In [ ]:
# ============================================================================
# 5. VISUALIZAÇÃO
# ============================================================================

print('\n📊 Gerando Visualizações...')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# ===== Subplot 1: Comparação das Curvas =====
ax1 = axes[0, 0]
ax1.plot(X_test, y_true, 'k-', linewidth=3, label='Função Real', alpha=0.7, zorder=1)
ax1.plot(X_test, y_pred, 'b-', linewidth=2.5, label='Predição Wang-Mendel', alpha=0.8, zorder=2)
ax1.scatter(X_train, y_train, color='red', s=80, zorder=5, 
           edgecolors='black', linewidths=1.5, label=f'Dados de Treino (n={n_samples})', alpha=0.8)
ax1.set_xlabel('x', fontsize=12, fontweight='bold')
ax1.set_ylabel('y', fontsize=12, fontweight='bold')
ax1.set_title('Aproximação Fuzzy vs Função Real', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.text(0.02, 0.98, f'R² = {r2:.4f}\nRMSE = {rmse:.4f}', 
        transform=ax1.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# ===== Subplot 2: Erro de Predição =====
ax2 = axes[0, 1]
erro = y_pred - y_true
ax2.plot(X_test, erro, 'r-', linewidth=2, alpha=0.7)
ax2.axhline(0, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
ax2.fill_between(X_test.flatten(), erro, 0, alpha=0.3, color='red')
ax2.set_xlabel('x', fontsize=12, fontweight='bold')
ax2.set_ylabel('Erro (Predição - Real)', fontsize=12, fontweight='bold')
ax2.set_title('Erro de Predição', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.text(0.02, 0.98, f'MAE = {mae:.4f}\nMáx Erro = {np.abs(erro).max():.4f}', 
        transform=ax2.transAxes, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))

# ===== Subplot 3: Histograma de Erros =====
ax3 = axes[1, 0]
ax3.hist(erro, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
ax3.axvline(0, color='red', linestyle='--', linewidth=2, label='Erro Zero')
ax3.set_xlabel('Erro', fontsize=12, fontweight='bold')
ax3.set_ylabel('Frequência', fontsize=12, fontweight='bold')
ax3.set_title('Distribuição dos Erros', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3, axis='y')

# ===== Subplot 4: Scatter Predição vs Real =====
ax4 = axes[1, 1]
ax4.scatter(y_true, y_pred, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax4.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 
        'r--', linewidth=2, label='Predição Perfeita', zorder=10)
ax4.set_xlabel('Valor Real', fontsize=12, fontweight='bold')
ax4.set_ylabel('Valor Predito', fontsize=12, fontweight='bold')
ax4.set_title('Predição vs Real', fontsize=14, fontweight='bold')
ax4.legend(fontsize=11)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n✅ Treinamento e Visualização Concluídos!')

In [ ]:
# ============================================================================
# 6. VISUALIZAR REGRAS
# ============================================================================
wm.system.plot_rule_matrix()